In [1]:
!pip install catboost

     |████████████████████████████████| 66.3MB 56kB/s 


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor

In [2]:
df = pd.read_csv('data/res_df.csv')

In [5]:
df.head()

,Porosity,Resistivity,Water_Saturation
0,0.12529,6.52,1.0
1,0.14511,6.52,1.0
2,0.14511,6.52,1.0
3,0.13481,8.00,1.0
4,0.13481,8.00,1.0


In [7]:
import random
random.seed(12) #12,close
a = random.uniform(0.8, 1.1)
m = random.uniform(1.6, 2.5)
n = random.uniform(1.6, 2.0)
rw = random.uniform(0.08, 0.12)

df['Archie_model_rand'] = ((a/(df['Porosity']**m))*(rw/df['Resistivity']))**(1/n) #randomization
df['Archie_model_sand'] = ((0.81/(df['Porosity']**1.7))*(0.1/df['Resistivity']))**(1/2.0) #sand
df['Archie_model_rocks'] = ((0.81/(df['Porosity']**1.7))*(0.1/df['Resistivity']))**(1/1.8) #rocks

df['Archie_model_rand'] = df['Archie_model_rand'].clip(0,1.0)
df['Archie_model_sand'] = df['Archie_model_sand'].clip(0,1.0)
df['Archie_model_rocks'] = df['Archie_model_rocks'].clip(0,1.0)

In [8]:
df.head()

,Porosity,Resistivity,Water_Saturation,Archie_model_rand,Archie_model_sand,Archie_model_rocks
0,0.12529,6.52,1.0,1.000000,0.651463,0.621171
1,0.14511,6.52,1.0,0.917616,0.575011,0.540721
2,0.14511,6.52,1.0,0.917616,0.575011,0.540721
3,0.13481,8.00,1.0,0.896623,0.552629,0.517387
4,0.13481,8.00,1.0,0.896623,0.552629,0.517387


In [9]:
X = df.drop(['Water_Saturation'],axis=1)
Y = df['Water_Saturation']

In [10]:
X_train,X_test,y_train,y_test = train_test_split(X,Y, test_size = 0.2,random_state=77)

CatBoost Model.

In [11]:
cr = CatBoostRegressor(iterations=3000, learning_rate=0.13, depth=12, random_seed=77, loss_function='RMSE', eval_metric='RMSE',verbose=0)

In [12]:
np.mean(cross_val_score(cr, X_test, y_test, cv=3, scoring='r2',verbose=0))
cr.fit(X_train,y_train)

In [ ]:
#cr.fit(X_train,y_train)

In [13]:
pred_cat = cr.predict(X_test)
mean_squared_error(y_test, pred_cat)

0.05406725180681803